<a href="https://colab.research.google.com/github/DhruvAgg10/Python-projects/blob/main/turboengine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
#import random forest regression
from sklearn.ensemble import RandomForestRegressor
#import test train split
from sklearn.model_selection import train_test_split

In [ ]:
dataset_paths = {
    "FD001": "/content/train_FD001.txt",
    "FD002": "/content/train_FD002.txt",
    "FD003": "/content/train_FD003.txt",
    "FD004": "/content/train_FD004.txt"
}

Without sep=' '

When you do not specify the sep parameter, pandas assumes the default separator (a comma , for CSV files).

Since your file is space-separated, pandas reads the entire row as a single column, resulting in only one column.

With sep=' '

By specifying sep=' ' (space separator), pandas correctly interprets spaces as column dividers and properly separates the data into multiple columns.

Column Headers for Train Data

Unit Number – Unique identifier for each engine

Time (cycles) – The number of operational cycles completed

Operational Setting 1

Operational Setting 2

Operational Setting 3 6-26. Sensor Measurements 1 to 26 – Various engine sensor readings

In [ ]:

# List to store processed datasets
datasets = []

column_names = [
    "unit_number", "time_in_cycles",
    "operational_setting_1", "operational_setting_2", "operational_setting_3"
] + [f"sensor_{i}" for i in range(1, 22)]

for dataset_id, path in dataset_paths.items():
    df = pd.read_csv(path, sep='\s+', header=None)
    df.columns = column_names
    df['dataset_id'] = dataset_id

    # Make unit_number unique by appending dataset_id
    df['unit_number'] = df['unit_number'].astype(str) + "_" + dataset_id

    # Compute RUL separately
    df['RUL'] = df.groupby("unit_number")["time_in_cycles"].transform(max) - df["time_in_cycles"]

    datasets.append(df)






<ipython-input-3-c3796b45c788>:18: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  df['RUL'] = df.groupby("unit_number")["time_in_cycles"].transform(max) - df["time_in_cycles"]
<ipython-input-3-c3796b45c788>:18: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  df['RUL'] = df.groupby("unit_number")["time_in_cycles"].transform(max) - df["time_in_cycles"]
<ipython-input-3-c3796b45c788>:18: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  df['RUL'

In [ ]:
# Merge all datasets into one
merged_data = pd.concat(datasets, ignore_index=True)

print(f"Merged dataset shape: {merged_data.shape}")

Merged dataset shape: (160359, 28)


In [ ]:

# Merge all datasets
merged_data = pd.concat(datasets, ignore_index=True)

print(merged_data[['unit_number', 'dataset_id', 'RUL']].head())


  unit_number dataset_id  RUL
0     1_FD001      FD001  191
1     1_FD001      FD001  190
2     1_FD001      FD001  189
3     1_FD001      FD001  188
4     1_FD001      FD001  187


In [ ]:
print(merged_data['unit_number'].unique()[:10])  # Check first 10 unique engines


['1_FD001' '2_FD001' '3_FD001' '4_FD001' '5_FD001' '6_FD001' '7_FD001'
 '8_FD001' '9_FD001' '10_FD001']


In [ ]:
dataset_mapping = {"FD001": 1, "FD002": 2, "FD003": 3, "FD004": 4}
merged_data["dataset_id"] = merged_data["dataset_id"].map(dataset_mapping)

In [ ]:
# Check failure cycles for Engine 1 in each dataset
for dataset_id in [1, 2, 3, 4]:
    engine_1_max_cycles = merged_data[merged_data['unit_number'] == f'1_{dataset_id}']['time_in_cycles'].max()
    print(f"Engine 1 in {dataset_id} failed after {engine_1_max_cycles} cycles.")

Engine 1 in 1 failed after nan cycles.
Engine 1 in 2 failed after nan cycles.
Engine 1 in 3 failed after nan cycles.
Engine 1 in 4 failed after nan cycles.


In [ ]:
from sklearn.model_selection import train_test_split

# Define feature columns (excluding unit_number, time_in_cycles, and RUL)
features = [col for col in merged_data.columns if col not in ['unit_number', 'time_in_cycles', 'RUL']]

# Split data into training and validation sets
X = merged_data[features]
y = merged_data['RUL']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training data shape: {X_train.shape}")
print(f"Validation data shape: {X_val.shape}")

Training data shape: (128287, 25)
Validation data shape: (32072, 25)


In [ ]:
from lightgbm import LGBMRegressor

# Train LightGBM model
lgbm_model = LGBMRegressor(n_estimators=200, learning_rate=0.05, max_depth=7, random_state=42)
lgbm_model.fit(X_train, y_train)

print("Model training completed! ✅")

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.012004 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3845
[LightGBM] [Info] Number of data points in the train set: 128287, number of used features: 25
[LightGBM] [Info] Start training from score 122.275110
Model training completed! ✅


In [12]:

# Initialize the Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)
print("Model training completed! ✅")

Model training completed! ✅


In [13]:
from xgboost import XGBRegressor

# Train XGBoost model
xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
xgb_model.fit(X_train, y_train)
print("Model training completed! ✅")


Model training completed! ✅


In [14]:
# Import required libraries
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Store trained models in a dictionary
models = {
    "LightGBM": lgbm_model,
    "XGBoost": xgb_model,
    "Random Forest": rf_model
}

In [15]:
# Evaluate each model
for model_name, model in models.items():
    # Predict on validation set
    y_pred = model.predict(X_val)

    # Compute MAE & RMSE
    mae = mean_absolute_error(y_val, y_pred)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))

    # Print results
    print(f"🔹 {model_name} Results:")
    print(f"   ✅ Mean Absolute Error (MAE): {mae:.2f}")
    print(f"   ✅ Root Mean Squared Error (RMSE): {rmse:.2f}\n")


🔹 LightGBM Results:
   ✅ Mean Absolute Error (MAE): 35.84
   ✅ Root Mean Squared Error (RMSE): 50.17

🔹 XGBoost Results:
   ✅ Mean Absolute Error (MAE): 36.28
   ✅ Root Mean Squared Error (RMSE): 50.64

🔹 Random Forest Results:
   ✅ Mean Absolute Error (MAE): 35.57
   ✅ Root Mean Squared Error (RMSE): 50.15



In [16]:
# Define test dataset paths
test_paths = {
    "FD001": "/content/test_FD001.txt",
    "FD002": "/content/test_FD002.txt",
    "FD003": "/content/test_FD003.txt",
    "FD004": "/content/test_FD004.txt"
}

# Define true RUL paths
rul_paths = {
    "FD001": "/content/RUL_FD001.txt",
    "FD002": "/content/RUL_FD002.txt",
    "FD003": "/content/RUL_FD003.txt",
    "FD004": "/content/RUL_FD004.txt"
}


In [17]:
# Process each test dataset separately
test_datasets = []

for dataset_id, path in test_paths.items():
    # Load test dataset
    test_df = pd.read_csv(path, sep='\s+', header=None)

    # Assign column names
    test_df.columns = column_names

    # Keep only the last cycle for each engine
    test_df = test_df.groupby("unit_number").last().reset_index()

    # Make unit_number unique by appending dataset_id
    test_df["unit_number"] = test_df["unit_number"].astype(str) + "_" + dataset_id

    # Convert dataset_id to numeric
    test_df["dataset_id"] = dataset_mapping[dataset_id]

    # Load true RUL values
    true_rul = pd.read_csv(rul_paths[dataset_id], header=None, names=["True_RUL"])

    # Add true RUL to test data
    test_df["True_RUL"] = true_rul["True_RUL"]

    # Append to list
    test_datasets.append(test_df)


In [20]:
# Merge all test datasets
merged_test_data = pd.concat(test_datasets, ignore_index=True)

print(f" Merged test dataset shape: {merged_test_data.shape}")


 Merged test dataset shape: (707, 28)


In [21]:
print(merged_test_data[['unit_number', 'dataset_id', 'operational_setting_1', 'operational_setting_2', 'operational_setting_3']].head(10))


  unit_number  dataset_id  operational_setting_1  operational_setting_2  \
0     1_FD001           1                -0.0006                 0.0004   
1     2_FD001           1                 0.0018                -0.0001   
2     3_FD001           1                -0.0016                 0.0004   
3     4_FD001           1                 0.0012                 0.0004   
4     5_FD001           1                -0.0013                -0.0004   
5     6_FD001           1                 0.0076                -0.0003   
6     7_FD001           1                 0.0016                -0.0001   
7     8_FD001           1                 0.0016                -0.0005   
8     9_FD001           1                -0.0003                 0.0004   
9    10_FD001           1                -0.0018                 0.0004   

   operational_setting_3  
0                  100.0  
1                  100.0  
2                  100.0  
3                  100.0  
4                  100.0  
5           

In [26]:
# Loop through models and evaluate on test data
for model_name, model in models.items():
    # Predict RUL
    merged_test_data[f"Predicted_RUL_{model_name}"] = model.predict(X_test)

    # Compute MAE & RMSE
    mae = mean_absolute_error(merged_test_data["True_RUL"], merged_test_data[f"Predicted_RUL_{model_name}"])
    rmse = np.sqrt(mean_squared_error(merged_test_data["True_RUL"], merged_test_data[f"Predicted_RUL_{model_name}"]))

    print(f"✅ {model_name} Test Accuracy Results:")
    print(f"   🔹 Mean Absolute Error (MAE): {mae:.2f}")
    print(f"   🔹 Root Mean Squared Error (RMSE): {rmse:.2f}\n")

✅ LightGBM Test Accuracy Results:
   🔹 Mean Absolute Error (MAE): 27.60
   🔹 Root Mean Squared Error (RMSE): 37.83

✅ XGBoost Test Accuracy Results:
   🔹 Mean Absolute Error (MAE): 28.92
   🔹 Root Mean Squared Error (RMSE): 38.83

✅ Random Forest Test Accuracy Results:
   🔹 Mean Absolute Error (MAE): 27.54
   🔹 Root Mean Squared Error (RMSE): 38.66



1️⃣ If MAE < 30 & RMSE < 50, model is industry-ready ✅

In [27]:

# Define a function to predict RUL based on user input
def predict_rul(op1, op2, op3, sensor_values, dataset_id):
    """
    Predicts Remaining Useful Life (RUL) given operational settings and sensor values.

    Parameters:
    - op1, op2, op3: Operational settings
    - sensor_values: List of 21 sensor readings
    - dataset_id: Numeric ID of dataset (1=FD001, 2=FD002, 3=FD003, 4=FD004)

    Returns:
    - Predicted RUL
    """
    # Ensure input matches the model's expected feature format
    input_data = np.array([[op1, op2, op3] + sensor_values + [dataset_id]])

    # Predict RUL using the trained LightGBM model
    predicted_rul = lgbm_model.predict(input_data)[0]

    return predicted_rul

In [30]:
# Example: Manually enter values
op1 = float(input("Enter Operational Setting 1: "))
op2 = float(input("Enter Operational Setting 2: "))
op3 = float(input("Enter Operational Setting 3: "))

# Get sensor values as a list
sensor_values = []
for i in range(1, 22):  # Assuming 21 sensors
    value = float(input(f"Enter Sensor {i} Value: "))
    sensor_values.append(value)

# Select dataset ID
dataset_id = int(input("Enter Dataset ID (1=FD001, 2=FD002, 3=FD003, 4=FD004): "))

# Predict RUL
predicted_rul = predict_rul(op1, op2, op3, sensor_values, dataset_id)

print(f" Predicted RUL: {predicted_rul:.2f} cycles")

Enter Operational Setting 1: 1
Enter Operational Setting 2: 2
Enter Operational Setting 3: -0.0027
Enter Sensor 1 Value: -0.0003
Enter Sensor 2 Value: 100.0
Enter Sensor 3 Value: 518.67
Enter Sensor 4 Value: 641.71
Enter Sensor 5 Value: 1588.45
Enter Sensor 6 Value: 1395.42
Enter Sensor 7 Value: 14.62
Enter Sensor 8 Value:  21.61
Enter Sensor 9 Value: 554.85
Enter Sensor 10 Value: 2388.01
Enter Sensor 11 Value: 9054.42
Enter Sensor 12 Value: 1.30
Enter Sensor 13 Value: 47.50
Enter Sensor 14 Value:  522.16
Enter Sensor 15 Value: 2388.06
Enter Sensor 16 Value: 8139.62
Enter Sensor 17 Value: 8.3803
Enter Sensor 18 Value: 0.03
Enter Sensor 19 Value: 393
Enter Sensor 20 Value: 2388
Enter Sensor 21 Value:  100.00
Enter Dataset ID (1=FD001, 2=FD002, 3=FD003, 4=FD004): 1
 Predicted RUL: 121.37 cycles


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


1 2 -0.0027 -0.0003 100.0 518.67 641.71 1588.45 1395.42 14.62 21.61 554.85 2388.01 9054.42 1.30 47.50 522.16 2388.06 8139.62 8.3803 0.03 393 2388 100.00 39.02 23.3916  